In [2]:
import os
import sys
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import table

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(repo_root)

from src import *

In [2]:
process_directory_real_data(os.getcwd())

FileNotFoundError: [Errno 2] No such file or directory: '/home/jayeun/core_classes/capstone_thesis/higher_order_ranking/exp/ex04/data'

In [25]:


def process_results(base_path):
    # Read the summary CSV files
    log_likelihood_df = pd.read_csv(os.path.join(base_path, 'log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()
    leadership_log_likelihood_df = pd.read_csv(os.path.join(base_path, 'leadership_log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()

    filtered_log_like = log_likelihood_df.drop(columns=['rep'])
    filtered_leader_like = leadership_log_likelihood_df.drop(columns=['rep'])
    
    log_like = subtract_columns(filtered_log_like, 3)
    leadership_log = subtract_columns(filtered_leader_like, 4)

    return log_like, leadership_log

def subtract_columns(df, compared_col):
    columns = df.columns.tolist()
    base_column = columns[compared_col] 
    print(base_column)
    for col in columns[2:]:
        
        df[col] = df[col] - df[base_column] 
    
    return df


def visualize_df(df, title):
 
    print(f"\n{title}\n")
    # styled_df = df.style.set_table_styles(
    # ).set_properties(**{
    #     'background-color': 'LightGray',
    #     'color': 'black',
    #     'border-color': 'black',
    #     'border-style': 'solid',
    #     'border-width': '1px',
    #     'text-align': 'left'
    # })
    

    # print(df.to_latex(escape=True))
    # display(styled_df)
    display(df)

In [4]:
unique_datasets = pd.read_csv(os.path.join(os.path.join(os.getcwd(), 'results'), 'log_likelihood_summary.csv'))['dataset'].unique()
dataset_info = pd.read_csv(os.path.join(repo_root, 'datasets', 'dataset_info.csv'))
filtered_dataset_info = dataset_info[dataset_info['dataset_id'].isin(unique_datasets)].set_index('dataset_id')

domain_order = ["Election", "Sport", "Preferences", "Other"]
filtered_dataset_info['Domain'] = pd.Categorical(filtered_dataset_info['Domain'], categories=domain_order, ordered=True)

filtered_dataset_info = filtered_dataset_info.sort_values(by=['Domain', 'K1', 'K2', 'Name'])

visualize_df(filtered_dataset_info, 'Datasets Tested')



Datasets Tested



,Domain,Name,N,M,K1,K2
dataset_id,,,,,,
18,Election,minneapolis_election,778,137482,1,3
28,Election,APA_election,6,203896,1,5
30,Election,UK_labor_party_vote,6,266,1,5
2,Election,debian,36,3352,1,9
58,Election,NSW_legislative_assembly_election,1384,13658150,1,10
8,Election,glasgow_city_council,209,188376,1,13
1,Election,irish_election,36,138011,1,14
7,Election,electoral_reform_society,30,35610,1,29
42,Sport,boxing,466,5594,1,17


In [27]:

log_like, leadership_log = process_results(os.path.join(os.getcwd(), 'results'))

def rename_df(df):
    merged_df = df.merge(dataset_info, left_on='dataset', right_on='dataset_id').drop(columns=['dataset_id','dataset'])
    # merged_df = merged_df.rename(columns={'Name': 'dataset_name'}).set_index('dataset_name')

    domain_order = ["Election", "Sport", "Preferences", "Other"]
    merged_df['Domain'] = pd.Categorical(merged_df['Domain'], categories=domain_order, ordered=True)
    merged_df = merged_df.sort_values(by=['Domain', 'K1', 'K2', 'Name'])
    merged_df = merged_df[['Domain', 'Name', 'BT', 'HO_BT', 'HOL_BT', 'Spring_Rank', 'Spring_Rank_Leadership', 'Page_Rank','Page_Rank_Leadership', 'Point_Wise']].set_index('Domain')

    return merged_df




HO_BT
HOL_BT


In [28]:

log_like = rename_df(log_like)
print(log_like)
visualize_df(log_like, 'log likelihood')

                                          Name          BT  HO_BT      HOL_BT  \
Domain                                                                          
Election                  minneapolis_election   -1.427299    0.0   -1.430777   
Election                          APA_election   -3.687974    0.0   -3.697385   
Election                   UK_labor_party_vote   -2.492293    0.0   -2.549922   
Election                                debian   -4.672714    0.0   -4.985351   
Election     NSW_legislative_assembly_election   -4.386355    0.0   -5.209932   
Election                  glasgow_city_council   -3.158094    0.0   -3.258237   
Election                        irish_election   -5.044229    0.0   -5.143064   
Election              electoral_reform_society   -6.177104    0.0   -6.195116   
Sport                                   boxing  -21.538988    0.0  -33.781858   
Sport                                     FIFA   -1.411075    0.0   -1.411982   
Sport                       

,Name,BT,HO_BT,HOL_BT,Spring_Rank,Spring_Rank_Leadership,Page_Rank,Page_Rank_Leadership,Point_Wise
Domain,,,,,,,,,
Election,minneapolis_election,-1.427299,0.0,-1.430777,-2.783171,-3.002387,-1.504377,-1.490002,-1.472192
Election,APA_election,-3.687974,0.0,-3.697385,-15.171521,-16.309253,-3.696012,-3.692933,-3.690112
Election,UK_labor_party_vote,-2.492293,0.0,-2.549922,-4.387135,-4.363823,-3.220820,-2.857342,-2.676718
Election,debian,-4.672714,0.0,-4.985351,-5.455923,-5.495389,-6.983254,-6.086496,-5.014611
Election,NSW_legislative_assembly_election,-4.386355,0.0,-5.209932,-4.430827,-4.723891,-4.911084,-4.685407,-4.498026
Election,glasgow_city_council,-3.158094,0.0,-3.258237,-3.273359,-3.494488,-3.266435,-3.199520,-3.193524
Election,irish_election,-5.044229,0.0,-5.143064,-5.625426,-5.717296,-5.196946,-5.077089,-5.082346
Election,electoral_reform_society,-6.177104,0.0,-6.195116,-6.307919,-6.431429,-6.236753,-6.274201,-6.184840
Sport,boxing,-21.538988,0.0,-33.781858,-23.283633,-29.227790,-31.821658,-26.381332,-23.168252


In [29]:
leadership_log = rename_df(leadership_log)
visualize_df(leadership_log, 'Leadership log likelihood')


Leadership log likelihood



,Name,BT,HO_BT,HOL_BT,Spring_Rank,Spring_Rank_Leadership,Page_Rank,Page_Rank_Leadership,Point_Wise
Domain,,,,,,,,,
Election,minneapolis_election,-0.926821,-0.002397,0.0,-1.529747,-1.610487,-0.988544,-0.975298,-0.968206
Election,APA_election,-1.390345,-0.003356,0.0,-5.365562,-4.904471,-1.389694,-1.389581,-1.389384
Election,UK_labor_party_vote,-1.184313,-0.006023,0.0,-1.700646,-1.697463,-1.312247,-1.222023,-1.188365
Election,debian,-1.249969,-0.042650,0.0,-1.368773,-1.272094,-2.004420,-1.719453,-1.403802
Election,NSW_legislative_assembly_election,-1.259555,-0.147084,0.0,-1.302655,-1.225069,-1.491958,-1.376108,-1.355591
Election,glasgow_city_council,-1.145459,-0.032338,0.0,-1.158389,-1.141853,-1.228527,-1.194760,-1.174335
Election,irish_election,-1.413646,-0.020173,0.0,-1.433378,-1.419383,-1.471437,-1.440121,-1.432245
Election,electoral_reform_society,-1.521415,-0.003062,0.0,-1.538380,-1.544886,-1.535947,-1.538078,-1.526059
Sport,boxing,-1.363940,-0.718950,0.0,-2.107475,-1.422653,-3.490589,-2.674647,-2.210012
